In [43]:
import requests
from pyquery import PyQuery as pq
import pandas as pd
import re


def get_title(e):
    e.map(lambda _, e: pq(e).text(pq(e).attr('title')))
    e.children().remove()


def get_buildings():
    response = requests.get(
        'https://cookieclicker.fandom.com/wiki/Buildings')
    html = pq(response.content)('.wikitable')

    price = html('td:nth-child(2) span')
    get_title(price)
    cps = html('td:nth-child(3) span')
    get_title(cps)

    buildings = pd.read_html(html('.wikitable').outer_html(), index_col=0)[0][
        ['Base Cost', 'Base CpS']]
    return buildings


def get_upgrades(buildings):
    response = requests.get(
        'https://cookieclicker.fandom.com/wiki/Upgrades')
    html = pq(response.content)('caption')
    upgrades = []
    for name in buildings.index:
        regex = re.compile(
            '\d+ (?={})'.format(name[:-1]), re.IGNORECASE)
        table = html(':contains("%s")' % name).parent()
        requirement = table('td:nth-child(3)')
        requirement.map(lambda i, e: pq(e).text(
            '= n' + name.split(' ')[0] + ' >= ' + regex.search(pq(e).text()).group(0)))

        price = table('td:nth-child(4) span:nth-child(2)')
        get_title(price)

        upgrades.append(pd.read_html(
            table.outer_html())[0][['Name', 'Unlock condition', 'Base price']])
    upgrades = pd.concat(upgrades, keys=buildings.index, axis=1)
    return upgrades


buildings = get_buildings()
upgrades = get_upgrades(buildings)
excel = pd.ExcelWriter('Cookie Clicker.xlsx')
buildings.to_excel(excel, sheet_name='Buildings')
upgrades.to_excel(excel, sheet_name='Upgrades')
excel.close()
